In [29]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

# from sklearn.cross_validation import StratifiedKFold, KFold
import random
from sklearn import preprocessing
import lightgbm as lgb
import gc
from scipy.stats import skew, boxcox
from bayes_opt import BayesianOptimization
from scipy import sparse
from sklearn.metrics import log_loss
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 2017

# Load Data

In [42]:
train = pd.read_pickle('../input/Data0220/' + 'train_2017-02-20-23-23.pkl')
test = pd.read_pickle('../input/Data0220/' + 'test_2017-02-20-23-23.pkl')

tr_feat_sparse = pd.read_pickle('../input/Data0220/' + 'tr_feat_sparse_2017-02-20-18-56.pkl')
tr_desc_sparse = pd.read_pickle('../input/Data0220/' + 'tr_desc_sparse_2017-02-20-18-56.pkl')
te_feat_sparse = pd.read_pickle('../input/Data0220/' + 'te_feat_sparse_2017-02-20-18-56.pkl')
te_desc_sparse = pd.read_pickle('../input/Data0220/' + 'te_desc_sparse_2017-02-20-18-56.pkl')

feat_sparse_cols = pd.read_pickle('../input/Data0220/' + 'feat_sparse_cols_2017-02-20-18-56.pkl')
desc_sparse_cols = pd.read_pickle('../input/Data0220/' + 'desc_sparse_cols_2017-02-20-18-56.pkl')
# test_listing = pd.read_pickle('../input/' + 'listing_id.pkl')

scaler_cols = pd.read_pickle('../input/Data0220/' + 'scaler_cols_2017-02-20-23-23.pkl')
features_to_use = pd.read_pickle('../input/Data0220/' + 'featurestouse_2017-02-20-23-23.pkl')
cal_clos = pd.read_pickle('../input/Data0220/' + 'cal_clos_2017-02-20-23-23.pkl')


train_y = pd.read_pickle('../input/Data0220/' + 'y_2017-02-20-23-23.pkl') 
print train.shape, test.shape

(49352, 91) (74659, 90)


In [43]:
full_data=pd.concat([train,test])

SSL = preprocessing.StandardScaler()
for col in scaler_cols:
    full_data[col], lam = boxcox(full_data[col] - full_data[col].min() + 1)
    full_data[col] = SSL.fit_transform(full_data[col].values.reshape(-1,1))    

In [44]:
# del_cols = ['high_frac', 'low_frac', 'medium_frac', 'manager_skill', 'count']
# del_feautres_to_use = list(set(features_to_use) - set(del_cols))

# print len(del_feautres_to_use),len(features_to_use)

train_size = train_y.shape[0]

train_X = np.hstack([full_data[features_to_use][:train_size], tr_desc_sparse.toarray(), tr_feat_sparse.toarray()])
test_X = np.hstack([full_data[features_to_use][train_size:], te_desc_sparse.toarray(), te_feat_sparse.toarray()])

print train_X.shape, test_X.shape

(49352, 474) (74659, 474)


In [45]:
index_low = train[train['interest_level'] == 'low']
index_medium = train[train['interest_level'] == 'medium']
index_high = train[train['interest_level'] == 'high']

In [47]:
index_tmp = (list(index_medium.index[:3839])
                    +list(index_high.index)
                    +list(index_low.index[:3839]))
# index_tmp= index_tmp.iloc(np.random.permutation(len(index_tmp)))
# index_tmp

random.shuffle(index_tmp)

In [48]:
train_X = train_X[index_tmp,:]
train_X.shape

(11517, 474)

In [49]:

y_map = {'low': 2, 'medium': 1, 'high': 0}
train_y = train.loc[index_tmp,'interest_level'].apply(lambda x: y_map[x]).values

train_y.shape

(11517,)

In [50]:
cv_dataset = lgb.Dataset(train_X, train_y)

# Tune LightGBM

### num_leaves, default=127, type=int, alias=num_leaf

    number of leaves in one tree



In [38]:
params = {
    'objective': 'multiclass',
    'num_class': 3,
    'learning_rate': 0.1
}
params['metrics'] = 'multi_logloss'

for x in [8,15,31,63,127,255]:
    params['num_leaves'] = x
    clf = lgb.cv(params, cv_dataset,
                num_boost_round = 100000, nfold =5,
#                 metrics = 'multi_logloss',
                early_stopping_rounds = 50)


    print x, ' \t', clf.values()[0][-1], len(clf.values()[0])

8  	0.40248754561
15  	0.403136003677
31  	0.404238582581
63  	0.407808235517
127  	0.413747282728
255  	0.418215763504


In [39]:
params['num_leaves'] = 8

### min_child_samples : int, default= 10
    Minimum number of data need in a child(leaf)

In [43]:
for x in [170,180,190,210,220,230,240,260,270]:
#     rgr = lgb.LGBMClassifier(learning_rate=0.1,                             
#                              n_estimators=100000,
#                              num_leaves=num_leaves,
#                              min_child_samples = x)

#     rgr.fit(X_train,y_train,
#             eval_set=[(X_val,y_val)],
#             eval_metric='multi_logloss',
#             early_stopping_rounds=50,
#             verbose = False)
    params['min_child_samples'] = x
    clf = lgb.cv(params, cv_dataset,
                num_boost_round = 100000, nfold =5,
#                 metrics = 'multi_logloss',
                early_stopping_rounds = 50)    

    print x, ' \t', clf.values()[0][-1], len(clf.values()[0])

170  	0.402026658153
180  	0.402316793701
190  	0.402046150242
210  	0.402081960279
220  	0.402418392348
230  	0.402003675997
240  	0.402353098622
260  	0.403137546434
270  	0.403066734422


In [44]:
params['min_child_samples'] = 230
# 5    	0.402637625694
# 10  	0.402637625694
# 20  	0.402637625694
# 40  	0.402533182893
# 50  	0.402753955237
# 60  	0.402259499882
# 70  	0.402136451189
# 80  	0.402399804569
# 90  	0.402513495789
# 100  	0.40248754561
# 110  	0.402281555538
# 120  	0.402461182092
# 130  	0.402056647423
# 140  	0.402437414772
# 150  	0.402321883011
# 160  	0.40264994971
# 170  	0.402026658153
# 180  	0.402316793701
# 190  	0.402046150242
# 200  	0.402114731857
# 210  	0.402081960279
# 220  	0.402418392348
# 230  	0.402003675997
# 240  	0.402353098622
# 250  	0.402856180969
# 260  	0.403137546434
# 270  	0.403066734422

### colsample_bytree : float
    Subsample ratio of columns when constructing each tree.

In [50]:
for x in [0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
#     rgr = lgb.LGBMClassifier(learning_rate=0.1,                             
#                              n_estimators=100000,
#                              num_leaves=num_leaves,
#                              min_child_samples = min_child_samples,
#                              colsample_bytree = x)

#     rgr.fit(X_train,y_train,
#             eval_set=[(X_val,y_val)],
#             eval_metric='multi_logloss',
#             early_stopping_rounds=50,
#             verbose = False)

    params['colsample_bytree'] = x
    clf = lgb.cv(params, cv_dataset,
                num_boost_round = 100000, nfold =5,
#                 metrics = 'multi_logloss',
                early_stopping_rounds = 50)    

    print x, ' \t', clf.values()[0][-1], len(clf.values()[0])
    

0.3  	0.402971847368
0.4  	0.402288144759
0.5  	0.401843532994
0.6  	0.401918978576
0.7  	0.402097982832
0.8  	0.402404102067
0.9  	0.402258495948


In [51]:
params['colsample_bytree'] = 0.5

### subsample : float
    Subsample ratio of the training instance.

In [62]:
params['subsample_freq'] = 1
for x in [0.5,0.6,0.7,0.8,0.9]:
#     rgr = lgb.LGBMClassifier(learning_rate=0.1,                             
#                              n_estimators=100000,
#                              num_leaves=num_leaves,
#                              min_child_samples = min_child_samples,
#                              colsample_bytree = colsample_bytree,
#                              subsample = x,
#                              subsample_freq=1)

#     rgr.fit(X_train,y_train,
#             eval_set=[(X_val,y_val)],
#             eval_metric='multi_logloss',
#             early_stopping_rounds=50,
#             verbose = False)

    params['subsample'] = x
    clf = lgb.cv(params, cv_dataset,
                num_boost_round = 100000, nfold =5,
#                 metrics = 'multi_logloss',
                early_stopping_rounds = 50) 
    

    print x, ' \t', clf.values()[0][-1], len(clf.values()[0])
    



0.5  	0.406650096173 409
0.6  	0.405685521077 388
0.7  	0.404601097921 399
0.8  	0.403444443167 456
0.9  	0.402641262952 434


In [63]:
params['subsample'] = 1

### max_bin : int, required
    Max number of discrete bin for features

In [65]:
for x in [15,31,63, 127, 255, 511, 1023, 2047]:
#     rgr = lgb.LGBMClassifier(learning_rate=0.1,                             
#                              n_estimators=100000,
#                              num_leaves=num_leaves,
#                              min_child_samples = min_child_samples,
#                              colsample_bytree = colsample_bytree,
#                              subsample = subsample,
#                              subsample_freq=1,
#                              max_bin = x )

#     rgr.fit(X_train,y_train,
#             eval_set=[(X_val,y_val)],
#             eval_metric='multi_logloss',
#             early_stopping_rounds=50,
#             verbose = False)


#     params['max_bin'] = x
    cv_dataset = lgb.Dataset(train_X, train_y, max_bin = x)
    clf = lgb.cv(params, cv_dataset,
                 num_boost_round = 100000, nfold =5,
                 early_stopping_rounds = 50)    

    print x, ' \t', clf.values()[0][-1], len(clf.values()[0])
 

15  	0.402199648117 467
31  	0.400999225417 582
63  	0.401839428426 526
127  	0.402102910186 480
255  	0.401843532994 530
511  	0.401728898913 512
1023  	0.402129273337 472
2047  	0.402452236718 482


In [66]:
params['max_bin'] = 582

In [28]:
def lgbm_cv(max_bin, num_leaves, min_child_samples, colsample_bytree, subsample, learning_rate=0.1):
    skf = list(KFold(len(train_y), 5))
    scores=[]
    for i, (train, val) in enumerate(skf):
        est=lgb.LGBMClassifier(learning_rate=0.1,
                               max_bin=int(max_bin),
                               num_leaves=int(num_leaves),
                               min_child_samples=int(min_child_samples),
                               colsample_bytree=colsample_bytree,
                               subsample=subsample,
                               subsample_freq = 1
                              )
 
        train_x_fold = train_X[train]
        train_y_fold = train_y[train]
        val_x_fold = train_X[val]
        val_y_fold = train_y[val]
        est.set_params( n_estimators=100000)
        est.fit(train_x_fold,
                train_y_fold,
                eval_set=[(val_x_fold, val_y_fold)],
                eval_metric='multi_logloss',
                early_stopping_rounds=50,
                verbose = False
               )
        val_y_predict_fold = est.predict_proba(val_x_fold)
        score = log_loss(val_y_fold, val_y_predict_fold)
        scores.append(score)
    return -np.mean(scores)


lgbm_BO = BayesianOptimization(lgbm_cv, 
                               {
                                'max_bin': (200,800),
                                'num_leaves': (15,63),
                                'min_child_samples' :(80,160),
                                'colsample_bytree': (0.5,1),
                                'subsample' : (0.8,1)})

lgbm_BO.maximize(init_points=10, n_iter=40)

Initialization
-----------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_bin |   min_child_samples |   num_leaves |   subsample | 
    1 | 02m50s |   -0.53476 |             0.9494 |  206.0155 |            111.6937 |      53.0017 |      0.9958 | 
    2 | 02m11s |   -0.53134 |             0.9097 |  710.0274 |             99.8716 |      25.7253 |      0.8223 | 
    3 | 01m12s |   -0.53298 |             0.5736 |  508.9968 |            144.0163 |      28.6098 |      0.9377 | 
    4 | 01m24s |   -0.53490 |             0.8441 |  683.6996 |             99.5012 |      60.9060 |      0.9267 | 
    5 | 01m06s |   -0.53391 |             0.5226 |  508.3731 |             85.1011 |      36.6520 |      0.8504 | 
    6 | 01m02s |   -0.53513 |             0.6736 |  319.5854 |            124.2871 |      61.2123 |      0.8560 | 
    7 | 01m09s |   -0.53263 |             0.6901 |  470.4126 |    

/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
/home/xujin/AI/anaconda2/lib/python2.7/site-packages/bayes_opt/helpers.py:95: RuntimeWarning: divide by zero encountered in true_divide
  z = (mean - y_max - xi)/std


Bayesian Optimization
-----------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_bin |   min_child_samples |   num_leaves |   subsample | 


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 0.00035908]), 'nit': 5, 'funcalls': 51}
  " state: %s" % convergence_dict)


   11 | 01m42s |   -0.53312 |             0.5305 |  797.2248 |             92.6381 |      25.8138 |      0.8091 | 


...

   46 | 01m55s |   -0.53223 |             0.7986 |  790.9825 |            139.1079 |      17.6317 |      0.8300 | 
   47 | 01m38s |   -0.53181 |             0.5268 |  468.2110 |            100.4549 |      20.0324 |      0.8654 | 
   48 | 01m55s |   -0.53173 |             0.7773 |  460.9678 |            157.3346 |      15.2557 |      0.9048 | 


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.0005743]), 'nit': 6, 'funcalls': 49}
  " state: %s" % convergence_dict)


   49 | 01m42s |   -0.53184 |             0.5113 |  260.9516 |            113.5624 |      15.8408 |      0.8203 | 


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 0.00112517]), 'nit': 3, 'funcalls': 46}
  " state: %s" % convergence_dict)


   50 | 01m40s |   -0.53239 |             0.7990 |  340.5197 |            105.1115 |      28.4028 |      0.9560 | 


In [30]:
gbm_bo_scores = pd.DataFrame([[s[0]['num_leaves'],
                               s[0]['min_child_samples'],
                               s[0]['max_bin'],
                               s[0]['colsample_bytree'],
                               s[0]['subsample'],
                               s[1]] for s in zip(lgbm_BO.res['all']['params'],lgbm_BO.res['all']['values'])],
                            columns = ['num_leaves',
                                       'min_child_samples',
                                       'max_bin',
                                       'colsample_bytree',
                                       'subsample',
                                       'score'])
gbm_bo_scores=gbm_bo_scores.sort_values('score',ascending=False)
gbm_bo_scores.head()

,num_leaves,min_child_samples,max_bin,colsample_bytree,subsample,score
20,16.458039,117.622103,478.697403,0.842639,0.820588,-0.531018
1,15.313144,116.494344,290.673175,0.762613,0.843922,-0.531225
13,16.691140,80.808810,202.206598,0.870567,0.861266,-0.531231
10,18.654455,158.680310,327.356657,0.754814,0.930391,-0.531267
9,18.192282,124.665623,333.804592,0.849373,0.912663,-0.531491


In [39]:
def lgbm_blend(params, train_x, train_y, test_x, fold, early_stopping_rounds=0):
    N_params = len(params)
    print ("Blend %d estimators for %d folds" % (N_params, fold))
    skf = KFold(n_splits=fold,random_state=seed)
    N_class = len(set(train_y))
    
    train_blend_x = np.zeros((train_x.shape[0], N_class*N_params))
    test_blend_x = np.zeros((test_x.shape[0], N_class*N_params))
    scores = np.zeros ((fold,N_params))
    best_rounds = np.zeros ((N_params))
    

    
    for j, param in enumerate(params):
        param['learning_rate']=0.01
        param['subsample_freq']=1
#         param['metrics'] = 'multi_logloss'  
        param['num_class'] = 3
        param['objective'] = 'multiclass'
        param['subsample_freq'] = 1
        
        print ("Model %d:" %(j+1)) 

        
        cv_dataset = lgb.Dataset(train_x, train_y, max_bin = param['max_bin'])
        clf = lgb.cv(param, cv_dataset,
                     num_boost_round = 100000, nfold =fold,
                     metrics = 'multi_logloss',
                     early_stopping_rounds = early_stopping_rounds) 
      
        best_round = len(clf.values()[0])
        print 'best_round',best_round
        best_rounds[j]=best_round
        
        all_round = best_round / (1 - 1. / fold)
        est_test_blend = lgb.train(param, cv_dataset,num_boost_round=int(all_round))

        test_blend_x[:,(j*N_class):(j+1)*N_class] = est_test_blend.predict(test_x)
        

        for i, (train_index, val_index) in enumerate(skf.split(train_x)):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x[train_index]
            train_y_fold = train_y[train_index]
            val_x_fold = train_x[val_index]
            val_y_fold = train_y[val_index]
            
            data_fold = lgb.Dataset(train_x_fold, train_y_fold, max_bin = param['max_bin'])
            est_train_blend = lgb.train(param, data_fold,num_boost_round=best_round)
            
            val_y_predict_fold = est_train_blend.predict(val_x_fold)
            score = log_loss(val_y_fold, val_y_predict_fold)
            print ("Score: ", score)
            scores[i,j]=score   
            
            train_blend_x[val_index, (j*N_class):(j+1)*N_class] = val_y_predict_fold
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            

        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x, scores,best_rounds)

In [52]:
lgb_params = [{
        'num_leaves': 8,
        'min_child_samples': 230,
        'colsample_bytree': 0.5,
        'subsample': 1,
        'max_bin': 582
    }]


(train_blend_x_gbm,
 test_blend_x_gbm,
 blend_scores_gbm,
 best_rounds_gbm) = lgbm_blend(lgb_params, 
                               train_X, train_y, 
                               test_X,
                               5,
                               500) #as the learning rate decreases the number of stopping rounds need to be increased



Blend 1 estimators for 5 folds
Model 1:
best_round 1762
Model 1 fold 1
('Score: ', 0.42060475512970935)
Model 1 fold 1 fitting finished in 27.545s
Model 1 fold 2
('Score: ', 0.43363531834175567)
Model 1 fold 2 fitting finished in 27.525s
Model 1 fold 3
('Score: ', 0.45222319176928)
Model 1 fold 3 fitting finished in 27.720s
Model 1 fold 4
('Score: ', 0.45110224941246674)
Model 1 fold 4 fitting finished in 27.795s
Model 1 fold 5
('Score: ', 0.41711606730175121)
Model 1 fold 5 fitting finished in 29.370s
Score for model 1 is 0.434936
Score for blended models is 0.434936


In [56]:
train_blend_x_gbm[:10]

array([[  4.37822603e-01,   5.25414695e-01,   3.67627023e-02],
       [  4.92365984e-01,   1.14699205e-01,   3.92934811e-01],
       [  5.36410955e-01,   3.93353677e-01,   7.02353680e-02],
       [  1.18684128e-01,   2.85832186e-01,   5.95483686e-01],
       [  3.71821146e-01,   4.19582087e-01,   2.08596767e-01],
       [  3.71477251e-04,   5.83382437e-03,   9.93794698e-01],
       [  1.53800383e-01,   4.68607746e-01,   3.77591871e-01],
       [  9.88012978e-01,   1.17017431e-02,   2.85278704e-04],
       [  5.30801793e-01,   3.68164446e-01,   1.01033761e-01],
       [  1.15524929e-03,   3.80137273e-03,   9.95043378e-01]])

In [55]:
train_y[:10]

array([0, 2, 0, 1, 0, 2, 0, 0, 0, 2])

In [38]:

now = datetime.now()

name_train_blend = '../output/train_blend_LightGBM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend = '../output/test_blend_LightGBM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



print (np.mean(blend_scores_gbm,axis=0))
print (np.mean(best_rounds_gbm,axis=0))
np.savetxt(name_train_blend,train_blend_x_gbm, delimiter=",")
np.savetxt(name_test_blend,test_blend_x_gbm, delimiter=",")

[ 0.52918957  0.52911925  0.52882423  0.52958864  0.52949003]
[ 5081.6  5483.8  5253.4  4707.8  4640.8]


In [45]:
now = datetime.now()
df = pd.read_json(open("../input/test.json", "r"))
labels2idx ={'high': 0, 'low': 2, 'medium': 1}
sub_name = '../output/sub_LightGBM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

sub = pd.DataFrame()
sub["listing_id"] = df["listing_id"]

y_test = np.zeros((df.shape[0], 3))

for N in range(3):
    y_test[:,N] = pd.DataFrame(test_blend_x_gbm).iloc[:,[x for x in range(test_blend_x_gbm.shape[1]) if x%3 == N]].mean(axis=1)
    
for label in ["high", "medium", "low"]:
    sub[label] = y_test[:, labels2idx[label]]
sub.to_csv(sub_name, index=False)

In [57]:
now = datetime.now()
df = pd.read_json(open("../input/test.json", "r"))
labels2idx ={'high': 0, 'low': 2, 'medium': 1}
sub_name = '../output/sub_LightGBM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

sub = pd.DataFrame()
sub["listing_id"] = df["listing_id"]

for label in ["high", "medium", "low"]:
    sub[label] = test_blend_x_gbm[:, labels2idx[label]]
sub.to_csv(sub_name, index=False)